In [38]:
import pandas as pd
from datebase import create_df_books
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [25]:
df = create_df_books()

In [28]:
selected_columns = ['title', 'authors', 'published_date', 'description', 'page_count', 'categories', 'language']
df = df[selected_columns]

In [39]:
df['main_category'] =df['categories'].apply(lambda x: x.split(' ')[0])

C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\1865943301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_category'] =df['categories'].apply(lambda x: x.split(' ')[0])


In [40]:
genres_df = pd.get_dummies(df.main_category)

In [41]:
autor = pd.get_dummies(df.authors)

In [42]:
language_df = pd.get_dummies(df.language)

In [43]:
features = pd.concat([ language_df, genres_df, autor], axis=1)

In [44]:
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features)

In [45]:
X = features
y = df[['title', 'main_category']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
n_neighbors = 10
knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(X_train)

In [57]:
def recommend_movies(title, n_recommendations=5):
    if title not in y_train['title'].values:
        return "Film nie znaleziony w bazie treningowej."
    
    idx = y_train[y_train['title'] == title].index[0]
    movie_features = X_train[idx]
    
    # Zmiana kształtu na (1, -1)
    movie_features = movie_features.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_features, n_neighbors=n_recommendations + 1)
    
    recommendations = []
    for i in range(1, len(indices[0])):
        recommendations.append(y_train.iloc[indices[0][i]])
    
    return recommendations

In [58]:
recommendations = recommend_movies("Harry Potter i Kamień Filozoficzny", n_recommendations=5)
for items in recommendations:
    print(items)

title            Zaćmienie
main_category             
Name: 39, dtype: object
title            Czterysta pięćdziesiąt jeden stopni Fahrenheita
main_category                                                   
Name: 35, dtype: object
title            Szatańskie wersety
main_category                      
Name: 62, dtype: object
title            Franklin zaprasza misia
main_category                           
Name: 11, dtype: object
title            Bardzo głodna gąsienica
main_category                           
Name: 5, dtype: object
